In [14]:
import pandas as pd
import paramiko
import os
import re
import json


In [5]:
# loop over all the files in data/struktureret/kv/valgresultater/kommuner 

files = os.listdir("data/struktureret/kv/valgresultater/kommune")
for file in files:
    # only keep the columns partier,procent_25,procent_21
    df = pd.read_csv(f"data/struktureret/kv/valgresultater/kommune/{file}")
    # only keep the columns partier,procent_25,procent_21   
    df = df[["partier","procent_25","procent_21"]]
    df.to_csv(f"data/struktureret/kv/valgresultater/kommune/{file}", index=False)

In [45]:
# load in all the csv files in data/kv21_resultater folder
path = "data/21_resultater/kv21_raw"

parti_info = json.load(open("data/partier.json"))
kommune_info = json.load(open("data/kommuner.json"))

parti_info = [parti for parti in parti_info if parti["listebogstav"] != "Æ" and parti["listebogstav"] != "M"]

super_df = pd.DataFrame()

csv_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith(".csv")]
for file in csv_files:
    df = pd.read_csv(file, sep=";")
    print(file)

    total_votes = df["Stemmetal"].sum()

    df = df.groupby(["Bogstavbetegnelse","Listenavn"]).agg({"Stemmetal":"sum"}).reset_index().sort_values(by="Stemmetal", ascending=False)
    df["Procent"] = df["Stemmetal"] / total_votes * 100

    def get_standard_party_name(row):
        key = str(row["Bogstavbetegnelse"]).strip().upper()
        for parti in parti_info:                  # ensure parti_info is a list of dicts
            if key == str(parti["listebogstav"]).strip().upper():
                # return standardized (name, bogstav)
                return parti["navn"], parti["bogstav"]
        # fallback: keep original values, but return a pair
        return row["Listenavn"], row["Bogstavbetegnelse"]

    df[["Listenavn", "bogstav"]] = df.apply(get_standard_party_name, axis=1, result_type="expand")  

    # get the kommune name from the file name by grabbing the part between the second and third underscore
    df['kommune_navn'] = file.split("_")[4]
    df['kommune_id'] = None
    df['kommune_dagi_id'] = None

    # get the kommune_id and the dagi_ud from kommune_info json
    for kommune in kommune_info:
        if kommune["kommune_navn"].lower() == df['kommune_navn'].iloc[0].lower():
            df['kommune_id'] = kommune["kommune_id"]
            df['kommune_dagi_id'] = kommune["kommune_dagi_id"]
            break


    #rename and reorder the columns: 
    #kommune_id,kommune_dagi_id,kommune_navn,listebogstav,partier,bogstav,procent_25,stemmer_25
    df = df[["kommune_id","kommune_dagi_id","kommune_navn","Bogstavbetegnelse","Listenavn","bogstav","Procent","Stemmetal"]]
    df.columns = ["kommune_id","kommune_dagi_id","kommune_navn","listebogstav","partier","bogstav","procent_21","stemmer_21"]

    super_df = pd.concat([super_df, df], ignore_index=True)
super_df
super_df.to_csv("data/21_resultater/kv21_parti_resultater.csv", index=False)

data/21_resultater/kv21_raw/Kommunalvalg_2021_Aarhus_06-11-2025 9.44.23.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Lolland_06-11-2025 9.38.35.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Tårnby_06-11-2025 9.43.8.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Tønder_06-11-2025 9.43.3.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Struer_06-11-2025 9.42.21.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Lejre_06-11-2025 9.38.23.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Ishøj_06-11-2025 9.37.18.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Kerteminde_06-11-2025 9.37.48.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Herlev_06-11-2025 9.34.33.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Albertslund_06-11-2025 9.29.31.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Brøndby_06-11-2025 9.30.46.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Dragør_06-11-2025 9.31.10.csv
data/21_resultater/kv21_raw/Kommunalvalg_2021_Helsingør_06-11-2025 9.34.27.csv
da

In [35]:
# load in all the csv files in data/kv21_resultater folde
path = "data/21_resultater/rv21_raw"
parti_info = json.load(open("data/partier.json"))
super_df = pd.DataFrame()

# In parti_info, drop the entry where listebogstav is "Æ" or M
parti_info = [parti for parti in parti_info if parti["listebogstav"] != "Æ" and parti["listebogstav"] != "M"]


csv_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith(".csv")]
for file in csv_files:
    print(f"Processing file: {file}")
    df = pd.read_csv(file, sep=";")


    total_votes = df["Stemmetal"].sum()

    df = df.groupby(["Bogstavbetegnelse","Listenavn"]).agg({"Stemmetal":"sum"}).reset_index().sort_values(by="Stemmetal", ascending=False)
    df["Procent"] = df["Stemmetal"] / total_votes * 100

    def get_standard_party_name(row):
        key = str(row["Bogstavbetegnelse"]).strip().upper()
        for parti in parti_info:   
            if key == str(parti["listebogstav"]).strip().upper():
                print(f"Mapping {key} to {parti['navn']} ({parti['bogstav']})")
                 # return standardized (name, bogstav)
                return parti["navn"], parti["bogstav"]
         # fallback: keep original values, but return a pair
        return row["Listenavn"], row["Bogstavbetegnelse"]

    df[["Listenavn", "bogstav"]] = df.apply(get_standard_party_name, axis=1, result_type="expand")  

    df['region_navn'] = file.split("_")[4]

    super_df = pd.concat([super_df, df], ignore_index=True)
super_df

super_df.to_csv("data/21_resultater/rv21_parti_resultater.csv", index=False)

Processing file: data/21_resultater/rv21_raw/Regionsrådsvalg_2021_Region Midtjylland_11-11-2025 10.20.9.csv
Mapping A to Socialdemokratiet (S)
Mapping V to Venstre (V)
Mapping C to Konservative (K)
Mapping F to SF (SF)
Mapping Ø to Enhedslisten (EL)
Mapping B to Radikale (R)
Mapping O to Dansk Folkeparti (DF)
Mapping K to Kristendemokraterne (KD)
Mapping I to Liberal Alliance (LA)
Mapping Å to Alternativet (ALT)
Processing file: data/21_resultater/rv21_raw/Regionsrådsvalg_2021_Region Syddanmark_11-11-2025 10.20.22.csv
Mapping V to Venstre (V)
Mapping A to Socialdemokratiet (S)
Mapping C to Konservative (K)
Mapping F to SF (SF)
Mapping Ø to Enhedslisten (EL)
Mapping O to Dansk Folkeparti (DF)
Mapping B to Radikale (R)
Mapping I to Liberal Alliance (LA)
Mapping K to Kristendemokraterne (KD)
Mapping Å to Alternativet (ALT)
Processing file: data/21_resultater/rv21_raw/Regionsrådsvalg_2021_Region Nordjylland_11-11-2025 10.20.15.csv
Mapping A to Socialdemokratiet (S)
Mapping V to Venstre 

In [ ]:
# # find the corresponding kommune_id and kommune_dagi_id from kommune_info
# for index, row in super_df.iterrows():
#     kommune_navn = row["kommune_navn"]
#     for kommune in kommune_info:
#         if kommune_navn.lower() == kommune["kommune_navn"].lower():
#             super_df.at[index, "kommune_id"] = kommune["kommune_id"]
#             super_df.at[index, "kommune_dagi_id"] = kommune["kommune_dagi_id"]
#             break

In [ ]:
# super_df

,kommune_id,kommune_dagi_id,kommune_navn,listebogstav,partier,bogstav,procent_21,stemmer_21
0,751,389194,Aarhus,A,Socialdemokratiet,S,28.739157,55295
1,751,389194,Aarhus,C,Konservative,K,14.592808,28077
2,751,389194,Aarhus,F,SF,SF,13.741470,26439
3,751,389194,Aarhus,V,Venstre,V,11.868318,22835
4,751,389194,Aarhus,Ø,Enhedslisten,EL,9.897975,19044
...,...,...,...,...,...,...,...,...
1113,710,389188,Favrskov,B,Radikale,R,3.781591,1007
1114,710,389188,Favrskov,D,Nye Borgerlige,D,3.537497,942
1115,710,389188,Favrskov,O,Dansk Folkeparti,DF,3.511210,935
1116,710,389188,Favrskov,L,Samlingspartiet Favrskov,L,1.186676,316


In [ ]:
#super_df.to_csv("data/kv21_resultater/parti_resultater.csv", index=False)

In [4]:
# FTP login information
host = "data.valg.dk"
port = 22
username = "Valg"
password = "Valg"

# Forbindelse til FTP-server
transport = paramiko.Transport((host, port))
transport.connect(username=username, password=password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [5]:
sftp.listdir('data/regionsrådsvalg-134-18-11-2025/geografi')

['Opstillingskreds-311020251900.json.hash',
 'Storkreds-311020251900.json.hash',
 'Storkreds-311020251900.json',
 'Valglandsdel-311020251900.json.hash',
 'Valglandsdel-311020251900.json',
 'Region-311020251900.json.hash',
 'Region-311020251900.json',
 'Kommune-311020251900.json.hash',
 'Kommune-311020251900.json',
 'Afstemningsomraade-311020251900.json.hash',
 'Afstemningsomraade-311020251900.json',
 'Opstillingskreds-311020251900.json']

In [6]:
with sftp.open('data/regionsrådsvalg-134-18-11-2025/verifikation/valgresultater/valgresultater-Regionsrådsvalg-Københavns_Kommune-3__Nordvest-190820250843.json') as f:
    text = f.read().decode('latin-1')  # or 'windows-1252'
    data = json.loads(text)

In [7]:
data

{'Valgdag': '18-11-2025',
 'Valgart': 'RegionsrÃ¥dsvalg',
 'FrigivelsesTidspunktUTC': '19-08-2025 06:43:57',
 'Storkreds': 'KÃ¸benhavn',
 'Storkredsnummer': 1,
 'Opstillingskreds': 'Ã\x98sterbro',
 'OpstillingskredsDagiId': 1,
 'Kommune': 'KÃ¸benhavn',
 'Kommunekode': 101,
 'AfstemningsomrÃ¥deNummer': 6,
 'AfstemningsomrÃ¥deDagiId': 706166,
 'AfstemningsomrÃ¥de': '3. Nordvest',
 'UgyldigeFremmÃ¸dteStemmerÃ\x85rsager': [],
 'UgyldigeBrevstemmerÃ\x85rsager': [],
 'Resultatart': 'IngenResultater',
 'AfgivneStemmer': 0,
 'GyldigeStemmer': 0,
 'UgyldigeStemmerUdoverBlanke': 0,
 'BlankeUgyldigeFremmÃ¸dteStemmer': 0,
 'BlankeUgyldigeBrevstemmer': 0,
 'AfgivneStemmerDifferenceFraForrigeValg': None,
 'GyldigeStemmerDifferenceFraForrigeValg': None,
 'UgyldigeStemmerDifferenceFraForrigeValg': None,
 'BlankeStemmerDifferenceFraForrigeValg': None,
 'AntalStemmeberettigedeVÃ¦lgereDifferenceFraForrigeValg': None,
 'AntalStemmeberettigedeVÃ¦lgere': 40000}

In [8]:
with sftp.open('data/regionsrådsvalg-134-18-11-2025/geografi/Afstemningsomraade-311020251900.json') as f:
    data = json.load(f)

In [9]:
data

[{'OprettetUtc': '11-08-2025 08:43:36',
  'OpdateretUtc': None,
  'Dagi_id': '709812',
  'Nummer': 5,
  'Navn': 'Langå, Svindinge Og Øksendrup',
  'Afstemningssted': {'Navn': 'Svindinge Forsamlingshus',
   'Adgangsadresse': {'Id': '0a3f508a-ffec-32b8-e044-0003ba298018',
    'Adressebetegnelse': 'Strædet 2, Svindinge, 5853 Ørbæk'}},
  'Kommunekode': 450,
  'Opstillingskredsnummer': 46,
  'Opstillingskreds_Dagi_id': '403614',
  'Type': 'Afstemningsområde'},
 {'OprettetUtc': '11-08-2025 08:43:36',
  'OpdateretUtc': None,
  'Dagi_id': '748590',
  'Nummer': 3,
  'Navn': 'Nymarken',
  'Afstemningssted': {'Navn': 'Nymarksskolen',
   'Adgangsadresse': {'Id': '0a3f5088-6fcb-32b8-e044-0003ba298018',
    'Adressebetegnelse': 'Nymarken 47, Hundslev, 5300 Kerteminde'}},
  'Kommunekode': 440,
  'Opstillingskredsnummer': 46,
  'Opstillingskreds_Dagi_id': '403614',
  'Type': 'Afstemningsområde'},
 {'OprettetUtc': '11-08-2025 08:43:36',
  'OpdateretUtc': None,
  'Dagi_id': '706574',
  'Nummer': 2,
  'N

In [10]:
afstemningssteder = []

for d in data:

    afstemningssteder.append({
        'dagi_id': d['Dagi_id'],
        'navn': d['Navn'],
        'nummer': d['Nummer'],
        'afstemningssted_navn': d['Afstemningssted']['Navn'],
        'kommune_id': d['Kommunekode'],
        'opstillingskreds_nummer': d['Opstillingskredsnummer'],
        'opstillingskreds_dagi_id': d['Opstillingskreds_Dagi_id'],
        'afstemningssted_adresse': d['Afstemningssted']['Adgangsadresse']['Adressebetegnelse']
    })

df_afstemningssteder = pd.DataFrame(afstemningssteder)
df_afstemningssteder      

,dagi_id,navn,nummer,afstemningssted_navn,kommune_id,opstillingskreds_nummer,opstillingskreds_dagi_id,afstemningssted_adresse
0,709812,"Langå, Svindinge Og Øksendrup",5,Svindinge Forsamlingshus,450,46,403614,"Strædet 2, Svindinge, 5853 Ørbæk"
1,748590,Nymarken,3,Nymarksskolen,440,46,403614,"Nymarken 47, Hundslev, 5300 Kerteminde"
2,706574,Kerteminde,2,Kerteminde Idrætscenter,440,46,403614,"Enggade 19, 5300 Kerteminde"
3,703760,Skellerup,8,Skellerup Forsamlingshus,450,46,403614,"Skellerup Byvej 16, 5540 Ullerslev"
4,703754,Nyborghallen,2,Nyborghallen,450,46,403614,"Halvej 1, 5800 Nyborg"
...,...,...,...,...,...,...,...,...
1309,707512,Sig,4,"Birgittegården, Rød Stue",573,54,403620,"Falkevej 4A, Sig, 6800 Varde"
1310,710690,Ølgod,30,Ølgod Hallen - Bemærk NYT afstemningssted,573,54,403620,"Skolegade 18, 6870 Ølgod"
1311,708972,Blåvand,19,Blåvand Borgerhus,573,54,403620,"Blåvandvej 30A, 6857 Blåvand"
1312,704694,Agerbæk,23,Agerbæk Hotel,573,54,403620,"Storegade 7, 6753 Agerbæk"


In [11]:
kommune_info = json.load(open("data/kommuner.json", "r", encoding="utf-8"))

In [18]:
#turnto dataframe
df_kommune_info = pd.DataFrame(kommune_info)
df_kommune_info

,kommune_id,kommune_navn,kommune_dagi_id,chart_id,table_id,map_id,region
0,101,København,389103.0,None,None,None,Østdanmark
1,147,Frederiksberg,389104.0,None,None,None,Østdanmark
2,151,Ballerup,389105.0,None,None,None,Østdanmark
3,153,Brøndby,389106.0,None,None,None,Østdanmark
4,155,Dragør,389107.0,None,None,None,Østdanmark
...,...,...,...,...,...,...,...
94,840,Rebild,389204.0,None,None,None,Nordjylland
95,846,Mariagerfjord,389202.0,None,None,None,Nordjylland
96,849,Jammerbugt,389205.0,None,None,None,Nordjylland
97,851,Aalborg,389206.0,None,None,None,Nordjylland


In [19]:
#turn kommune_id to an int
df_kommune_info['kommune_id'] = df_kommune_info['kommune_id'].astype(int)
#join region on df_afstemningssteder
df_merged = df_afstemningssteder.merge(df_kommune_info[['kommune_id', 'region']], on='kommune_id', how='left')


In [25]:
df_merged

,dagi_id,navn,nummer,afstemningssted_navn,kommune_id,opstillingskreds_nummer,opstillingskreds_dagi_id,afstemningssted_adresse,region
0,709812,"Langå, Svindinge Og Øksendrup",5,Svindinge Forsamlingshus,450,46,403614,"Strædet 2, Svindinge, 5853 Ørbæk",Syddanmark
1,748590,Nymarken,3,Nymarksskolen,440,46,403614,"Nymarken 47, Hundslev, 5300 Kerteminde",Syddanmark
2,706574,Kerteminde,2,Kerteminde Idrætscenter,440,46,403614,"Enggade 19, 5300 Kerteminde",Syddanmark
3,703760,Skellerup,8,Skellerup Forsamlingshus,450,46,403614,"Skellerup Byvej 16, 5540 Ullerslev",Syddanmark
4,703754,Nyborghallen,2,Nyborghallen,450,46,403614,"Halvej 1, 5800 Nyborg",Syddanmark
...,...,...,...,...,...,...,...,...,...
1309,707512,Sig,4,"Birgittegården, Rød Stue",573,54,403620,"Falkevej 4A, Sig, 6800 Varde",Syddanmark
1310,710690,Ølgod,30,Ølgod Hallen - Bemærk NYT afstemningssted,573,54,403620,"Skolegade 18, 6870 Ølgod",Syddanmark
1311,708972,Blåvand,19,Blåvand Borgerhus,573,54,403620,"Blåvandvej 30A, 6857 Blåvand",Syddanmark
1312,704694,Agerbæk,23,Agerbæk Hotel,573,54,403620,"Storegade 7, 6753 Agerbæk",Syddanmark


In [27]:
# for each kommune dagi id generate a csv file with the afstemningssteder for that kommune
output_dir = 'data/struktureret/rv/valgresultater/afstemningssteder/'

os.makedirs(output_dir, exist_ok=True)

num = 0
for region, group in df_merged.groupby('region'):
    region_navn = group['region'].iloc[0].lower().replace(' ', '_')
    output_file = os.path.join(output_dir, f'{region_navn}_afstemningsområde.csv')
    group['største_parti'] = ''  # placeholder column for største parti
    group.to_csv(output_file, index=False)
    num += 1
print(f'Wrote afstemningssteder for {num} regioner to {output_dir}')

Wrote afstemningssteder for 4 regioner to data/struktureret/rv/valgresultater/afstemningssteder/


In [29]:
# also generate a file for each kommune in the folder data/struktureret/kv/valgresultater/kommune/
output_dir_kommune = 'data/struktureret/rv/valgresultater/region/'

os.makedirs(output_dir_kommune, exist_ok=True)
# the file should have the columns: kommune_id, kommune_dagi_id, kommune_navn, parti, procent_25, stemmer_25, procent_21, stemmer_21

for region, group in df_merged.groupby('region'):
    region_navn = group['region'].iloc[0].lower().replace(' ', '_')
    output_file = os.path.join(output_dir_kommune, f'{region_navn}.csv')
    df_kommune = pd.DataFrame([{
        'region_navn': group['region'].iloc[0],
        'partier': '',
        'procent_25': '',
        'stemmer_25': '',
        'procent_21': '',
        'stemmer_21': ''
    }])
    df_kommune.to_csv(output_file, index=False)

In [16]:
regioner_geo = json.load(open("data/shapes/regioner_afstemningsområder.geojson"))

In [18]:
# the file has shapes for five regions and their afstemningsområder
old_regioner = ['Region Hovedstaden', 'Region Sjælland', 'Region Syddanmark', 'Region Midtjylland', 'Region Nordjylland']
new_regioner = ['Region Østdanmark', 'Region Syddanmark', 'Region Midtjylland', 'Region Nordjylland']

# split the geojson into separate files for each new region
for region in new_regioner:
    region_geo = {
        "type": "FeatureCollection",
        "features": []
    }
    for feature in regioner_geo['features']:
        if region == 'Region Østdanmark':
            if feature['properties']['regionsnavn'] in ['Region Hovedstaden', 'Region Sjælland']:
                region_geo['features'].append(feature)
        else:
            if feature['properties']['regionsnavn'] == region:
                region_geo['features'].append(feature)
    with open(f"data/shapes/{region.lower().replace(' ', '_')}_afstemningsområder.geojson", "w") as f:
        json.dump(region_geo, f)

In [3]:
# turn data/kommuner.json into a dataframe
df_kommuner = pd.DataFrame(kommune_info)

In [6]:
df_kommuner.to_clipboard()

In [ ]:
# loop over each kommune in kommune info and print the kommune_id and kommune_navn
for region, group in df_merged.groupby('region'):
    # create a dataframe with the columns    kommune_id kommune_navn  andel_af_afstemningssteder_talt  borgmester
    df = pd.DataFrame({
        'region_navn': [group['region']],
        'andel_af_afstemningssteder_talt': 0,
        'Regionsrådsforperson': "Ikke afgjort"
    })
    # save the dataframe to a csv file in data/struktureret/kv/valgresultater/status/
    region_navn = group['region'].iloc[0].lower().replace(' ', '_')

    #df.to_csv(f"data/struktureret/rv/valgresultater/status/{region_navn}_status.csv", index=False)

                                         region_navn  \
0  19      Midtjylland
20      Midtjylland
21    ...   

   andel_af_afstemningssteder_talt Regionsrådsforperson  
0                                0         Ikke afgjort  
                                         region_navn  \
0  62      Nordjylland
63      Nordjylland
64    ...   

   andel_af_afstemningssteder_talt Regionsrådsforperson  
0                                0         Ikke afgjort  
                                         region_navn  \
0  0       Syddanmark
1       Syddanmark
2       ...   

   andel_af_afstemningssteder_talt Regionsrådsforperson  
0                                0         Ikke afgjort  
                                         region_navn  \
0  33      Østdanmark
34      Østdanmark
35      ...   

   andel_af_afstemningssteder_talt Regionsrådsforperson  
0                                0         Ikke afgjort  


In [ ]:
# for each kommune dagi id generate a csv file with the afstemningssteder for that kommune
output_dir = 'data/struktureret/rv/valgresultater/afstemningssteder/'

os.makedirs(output_dir, exist_ok=True)

num = 0
for kommune_id, group in df_afstemningssteder.groupby('kommune_id'):
    kommune_navn = group['kommune_navn'].iloc[0].lower().replace(' ', '_')
    output_file = os.path.join(output_dir, f'{kommune_id}_{kommune_navn}_afstemningsområde.csv')
    group['største_parti'] = ''  # placeholder column for største parti
    group.to_csv(output_file, index=False)
    num += 1
print(f'Wrote afstemningssteder for {num} kommuner to {output_dir}')